
# How to build a fast AI-surrogate starting from a slow simulator

"""
Simulator: Given circuit parameters, outputs frequency response
Real simulator: 10 seconds per evaluation
Goal: ML model that predicts in <1ms
"""

Slow simulator (you'll make a fake one)
def slow_circuit_simulator(R, L, C, frequencies):
    """
    Computes frequency response of RLC circuit
    Pretend this is expensive (add sleep to simulate)
    """
    import time
    time.sleep(0.1)  # Simulate expensive computation
    
    # Simple RLC transfer function
    # TODO: Implement H(jω) = 1 / (1 + jωRC - ω²LC)
    pass

Your tasks:
1. Generate training data efficiently (Latin Hypercube Sampling)
2. Build surrogate model (predict response from R, L, C, freq)
3. Compare speed: surrogate vs simulator
4. Validate accuracy on test set
5. Implement uncertainty estimation (bonus)


First, we'll be generating some data coming from a fake slow simulator:

In [1]:
# Importing useful libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import pandas as pd

from scipy import signal
import numpy as np

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") 
print(f"Using device: {device}")

Using device: cuda


let's define the fake simulator to generate some data:

In [3]:
# Slow simulator (you'll make a fake one)
def slow_circuit_simulator(R, L, C, frequencies):
    """
    Computes frequency response of RLC circuit
    Pretend this is expensive (add sleep to simulate)
    """
    #import time
    #time.sleep(0.1)  # Simulate expensive computation
    
    # Simple RLC transfer function
    # TODO: Implement H(jω) = 1 / (1 + jωRC - ω²LC)
    num = [1]
    den = [L*C, R*C, 1]
    w = 2 * np.pi * frequencies  # Convert Hz to rad/s

    # Create TransferFunction object
    system = signal.TransferFunction(num, den)
    
    # Evaluate frequency response
    w, H = signal.freqresp(system, w)

    # Extract real and imaginary parts
    real_part = np.real(H)
    imag_part = np.imag(H)
    
    return real_part, imag_part

now we can feed the fake simulator with a dataset that would try to fill the whole operational space. In order to do that, we'll be using LHS (Latin Hypercube Sampling). Let's create 50 different circuits (with different R, L, C values) and for all of them simulate the output at 1000 different frequencies:

In [4]:
from scipy.stats import qmc

# Latin Hypercube Sampling for 3 parameters and frequencies: R, L, C, w

sampler = qmc.LatinHypercube(d=3)
l_bounds = [1, 1e-6, 1e-9]  # R: 1-100 Ohm, L: 1uH-10mH, C: 1nF-10uF
u_bounds = [100, 10e-3, 10e-6]  # R: 1-100 Ohm, L: 1uH-10mH, C: 1nF-10uF
n_circuits = 50

circuit_parameters = sampler.random(n=n_circuits)

circuit_param = pd.DataFrame(circuit_parameters, columns=['R', 'L', 'C'])

In [5]:
w_values = np.logspace(3, 6, num=1000)  # Frequencies from 1kHz to 1MHz

The inputs are uniformally distributed, we can proceed generating the outputs through the fake simulator

In [6]:
output_real = []
output_imag = []
data = {
    "R": [],
    "L": [],    
    "C": [],
    "w": [],    
    "Real": [],
    "Imag": []
}

for index, row in circuit_param.iterrows():
    R = row['R']
    L = row['L']
    C = row['C']

    real_part, imag_part = slow_circuit_simulator(R, L, C, w_values)

    output_real.append(real_part)
    output_imag.append(imag_part)   

    for i in range(len(w_values)): 
        data["R"].append(R)
        data["L"].append(L)
        data["C"].append(C)
        data["w"].append(w_values[i])
        data["Real"].append(real_part[i])
        data["Imag"].append(imag_part[i])

In [7]:
data_df = pd.DataFrame(data)

Okay, now we have generated the outputs from the fake slow simulator, we can start using the data to train a model.

In [9]:
from sklearn.model_selection import train_test_split

features_names = ["R", "L", "C", "w"]
output_names = ["Real", "Imag"]

X = data_df[features_names].values
y = data_df[output_names].values

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, shuffle=True)

now let's create batches using a DataLoader function: 

In [10]:
from torch.utils.data import TensorDataset, DataLoader

train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32), 
                              torch.tensor(y_train, dtype=torch.float32))

test_dataset = TensorDataset(torch.tensor(X_test, dtype=torch.float32),
                                torch.tensor(y_test, dtype=torch.float32))

train_loader = DataLoader(
    train_dataset,
    batch_size=64, 
    shuffle=True, 
    pin_memory=True,
    num_workers=4,
    persistent_workers=True
)

test_loader = DataLoader(test_dataset,
                        batch_size=64,
                        shuffle=False,
                        pin_memory=True,
                        num_workers=4,
                        persistent_workers=True
                        )

now we have the data loaders ready, let's define a first architecture for the model surrogate:

In [11]:
class SurrogateModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SurrogateModel, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        return self.model(x)
    
input_dim = len(features_names)  # 4: R, L, C, w
output_dim = len(output_names)  # 2: Real, Imag

now let's choose an optimizer. For this optimization problem, I would use an STD (Stochastic Gradient Descent) algorithm. Thus, we can select Adam since it is a Gradient-Descent based optimization algorithm:

In [15]:
import torch.optim as optim
import mlflow

model = SurrogateModel(input_dim, hidden_dim=128, output_dim=output_dim).to(device=device)
optimizer = optim.Adam(model.parameters(), lr = 0.001)

criterion = nn.MSELoss()

with mlflow.start_run():
    
    n_epochs = 70

    mlflow.log_param("model_type", "SurrogateModel")
    mlflow.log_param("optimizer", "Adam")
    mlflow.log_param("learning_rate", 0.001)
    mlflow.log_param("batch_size", 64)
    mlflow.log_param("n_epochs", n_epochs)
 
    for epoch in range(n_epochs):
        
        model.train()
        epoch_training_loss = 0.0
        epoch_validation_loss = 0.0
        history = {
            "train_loss": [],
            "val_loss": []}
        
        for batch_x, batch_y in train_loader:

            batch_x = batch_x.to(device=device)
            batch_y = batch_y.to(device=device)

            # Clear gradients
            optimizer.zero_grad()

            # Forward pass
            predictions = model(batch_x)
            loss = criterion(predictions, batch_y)
            loss.backward()
            optimizer.step()

            epoch_training_loss += loss.item() * batch_x.size(0)

        with torch.no_grad():
            model.eval()
            for val_x, val_y in test_loader:
                val_x = val_x.to(device=device)
                val_y = val_y.to(device=device)

                val_predictions = model(val_x)
                val_loss = criterion(val_predictions, val_y)

                epoch_validation_loss += val_loss.item() * val_x.size(0)

        epoch_training_loss /= len(train_loader.dataset)
        epoch_validation_loss /= len(test_loader.dataset)

        history["train_loss"].append(epoch_training_loss)
        history["val_loss"].append(epoch_validation_loss)

        if (epoch+1) % 10 == 0 or epoch == 0:
            print(f"Epoch [{epoch+1}/{n_epochs}], "
                  f"Train Loss: {epoch_training_loss:.6f}, "
                  f"Val Loss: {epoch_validation_loss:.6f}")
            
        mlflow.log_metric("train_loss", epoch_training_loss, step=epoch)
        mlflow.log_metric("val_loss", epoch_validation_loss, step=epoch)


Epoch [1/70], Train Loss: 286037.808475, Val Loss: 1.864548
Epoch [10/70], Train Loss: 1767.984526, Val Loss: 18.394371
Epoch [20/70], Train Loss: 0.003217, Val Loss: 0.003174
Epoch [30/70], Train Loss: 0.000730, Val Loss: 0.000552
Epoch [40/70], Train Loss: 0.000000, Val Loss: 0.000000
Epoch [50/70], Train Loss: 0.000000, Val Loss: 0.000000
Epoch [60/70], Train Loss: 0.000000, Val Loss: 0.000000
Epoch [70/70], Train Loss: 0.000000, Val Loss: 0.000000


In [18]:
import os

save_dir = "model"
os.makedirs(save_dir, exist_ok=True)
save_path = os.path.join(save_dir, "surrogate_model.pth")
torch.save(model.state_dict(), save_path)